In [3]:
import pandas as pd
import spikeinterface as si
import spikeinterface.extractors as se
from pyscan.session_utils import gs_to_df
from pyscan.preprocessing.np2_preprocessing import sort_np2

# Required spreadsheet columns:
# trial_name: name of the trial
# path: path to the trial
# probe_type: type of probe used
# num_channels: number of channels
# Include: 'Y' if the trial should be included in the sorting


# Load sheet
sheet = gs_to_df('https://docs.google.com/spreadsheets/d/1_Xs5i-rHNTywV-WuQ8-TZliSjTxQCCqGWOD2AL_LIq0/edit#gid=0')
path_to_data = '/home/isabella/Documents/isabella/jake/recording_data/'
sorting_suffix = 'sorting_ks4'
sheet['path'] = path_to_data + sheet['path']

# Select only NP2 data to be sorted
sheet_inc = sheet[sheet['Include'] == 'Y']
sheet_inc = sheet_inc[sheet_inc['probe_type'] == 'NP2_openephys']

trial_list = sheet_inc['trial_name'].to_list()
session_list = np.unique([f"{i.split('_')[0]}_{i.split('_')[1]}" for i in trial_list])
recording_list = [[] for _ in range(len(session_list))]

# Collect recordings into sessions and preprocess
for i, session in enumerate(session_list):
    for trial in trial_list:
        if session in trial:
            base_folder = sheet_inc[sheet_inc['trial_name'] == trial]['path'].tolist()[0]
            num_channels = int(sheet_inc[sheet_inc['trial_name'] == trial]['num_channels'].tolist()[0])
            electrode_type = sheet_inc[sheet_inc['trial_name'] == trial]['probe_type'].tolist()[0]
            print(f"{base_folder}/{trial}")
            recording = se.read_openephys(folder_path=f"{base_folder}/{trial}", stream_id = '0')
            recording_list[i].append([recording,
                                     trial, 
                                     base_folder, 
                                     electrode_type])

# Concatenate over a single session and sort
for recording in recording_list:
    session = pd.DataFrame(recording)
    recordings_concat = si.concatenate_recordings(session.iloc[:,0].to_list())
    print(f'Sorting {recordings_concat}')
    
    sorting = sort_np2(recording = recordings_concat, 
			recording_name = session.iloc[0,1], 
			base_folder = session.iloc[0,2],
			sorting_suffix = sorting_suffix)
    session.to_csv(f'{session.iloc[0,2]}/{session.iloc[0,1][:6]}_{sorting_suffix}/session.csv') #save session trial info to .csv

/home/isabella/Documents/isabella/jake/recording_data/r1503/2024-03-15/240315_r1503_open-field_1
Sorting ConcatenateSegmentRecording: 384 channels - 30.0kHz - 1 segments - 18,613,123 samples 
                             620.44s (10.34 minutes) - int16 dtype - 13.31 GiB
Loading recording with SpikeInterface...
number of samples: 18613123
number of channels: 384
numbef of segments: 1
sampling rate: 30000.0
dtype: int16


/data/isabella/jake/ephys_analysis/Kilosort/kilosort/io.py:498: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  X[:, self.nt : self.nt+nsamp] = torch.from_numpy(data).to(self.device).float()


Preprocessing filters computed in  1.37s; total  1.37s

computing drift
Re-computing universal templates from data.


100%|██████████| 311/311 [03:37<00:00,  1.43it/s]


drift computed in  220.70s; total  222.06s

Extracting spikes using templates
Re-computing universal templates from data.


...

100%|██████████| 311/311 [01:18<00:00,  3.95it/s]


1017214 spikes extracted in  78.95s; total  557.30s

Final clustering


100%|██████████| 96/96 [00:40<00:00,  2.38it/s]


375 clusters found, in  40.38s; total  597.69s

Merging clusters
350 units found, in  1.62s; total  599.30s

Saving to phy and computing refractory periods
170 units found with good refractory periods

Total runtime: 600.60s = 00:10:0 h:m:s
kilosort4 run time 600.98s
Recording sorted!
 KS4 found 350 units

Sorting saved to /home/isabella/Documents/isabella/jake/recording_data/r1503/2024-03-15/240315_sorting_ks4/sort



/data/isabella/jake/ephys_analysis/spikeinterface/src/spikeinterface/core/basesorting.py:239: UserWarning: The registered recording will not be persistent on disk, but only available in memory
  warnings.warn("The registered recording will not be persistent on disk, but only available in memory")


In [28]:
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.widgets as sw

recording_path = '/data/isabella/jake/recording_data/NP2 data/2024-03-15/test/2024-03-15_13-05-49'
sorting_path = '/data/isabella/jake/recording_data/NP2 data/2024-03-15/test/kilosort4'

recording = se.read_openephys(folder_path=recording_path, stream_id = '0')
sorting = se.read_phy(sorting_path, exclude_cluster_groups=['noise', 'mua'])


import spikeinterface.postprocessing as sp
sorting_analyzer = si.create_sorting_analyzer(sorting=sorting, recording=recording)
sorting_analyzer.compute('random_spikes')
sorting_analyzer.compute('waveforms')
sorting_analyzer.compute_one_extension('templates')
si.postprocessing.compute_template_metrics(sorting_analyzer)
unit_locations = sorting_analyzer.compute(input="unit_locations", method="monopolar_triangulation")

sw.plot_rasters(sorting, time_range=[0, 10])



Loading recording with SpikeInterface...
number of samples: 18613123
number of channels: 384
numbef of segments: 1
sampling rate: 30000.0
dtype: int16
Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
Using GPU for PyTorch computations. Specify `device` to change this.


TypeError: string indices must be integers